# Projeto Airbnb Rio - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns

###Praticando o exercicio feito pela Hashtag

### Contexto

No Airbnb, qualquer pessoa que tenha um quarto ou um imóvel de qualquer tipo (apartamento, casa, chalé, pousada, etc.) pode ofertar o seu imóvel para ser alugado por diária.

Você cria o seu perfil de host (pessoa que disponibiliza um imóvel para aluguel por diária) e cria o anúncio do seu imóvel.

Nesse anúncio, o host deve descrever as características do imóvel da forma mais completa possível, de forma a ajudar os locadores/viajantes a escolherem o melhor imóvel para eles (e de forma a tornar o seu anúncio mais atrativo)

Existem dezenas de personalizações possíveis no seu anúncio, desde quantidade mínima de diária, preço, quantidade de quartos, até regras de cancelamento, taxa extra para hóspedes extras, exigência de verificação de identidade do locador, etc.

### Nosso objetivo

Construir um modelo de previsão de preço que permita uma pessoa comum que possui um imóvel possa saber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesmas características) ou não.

### O que temos disponível, inspirações e créditos

As bases de dados foram retiradas do site kaggle: https://www.kaggle.com/allanbruno/airbnb-rio-de-janeiro

Elas estão disponíveis para download abaixo da aula (se você puxar os dados direto do Kaggle pode ser que encontre resultados diferentes dos meus, afinal as bases de dados podem ter sido atualizadas).

Caso queira uma outra solução, podemos olhar como referência a solução do usuário Allan Bruno do kaggle no Notebook: https://www.kaggle.com/allanbruno/helping-regular-people-price-listings-on-airbnb

Você vai perceber semelhanças entre a solução que vamos desenvolver aqui e a dele, mas também algumas diferenças significativas no processo de construção do projeto.

- As bases de dados são os preços dos imóveis obtidos e suas respectivas características em cada mês.
- Os preços são dados em reais (R$)
- Temos bases de abril de 2018 a maio de 2020, com exceção de junho de 2018 que não possui base de dados

### Expectativas Iniciais

- Acredito que a sazonalidade pode ser um fator importante, visto que meses como dezembro costumam ser bem caros no RJ
- A localização do imóvel deve fazer muita diferença no preço, já que no Rio de Janeiro a localização pode mudar completamente as características do lugar (segurança, beleza natural, pontos turísticos)
- Adicionais/Comodidades podem ter um impacto significativo, visto que temos muitos prédios e casas antigos no Rio de Janeiro

Vamos descobrir o quanto esses fatores impactam e se temos outros fatores não tão intuitivos que são extremamente importantes.

### Importar Bibliotecas e Bases de Dados

In [7]:
import pandas as pd
import pathlib    # permite identificar uma pasta e percorrer todas as pastas


In [8]:
#entro na pasta a partir desse caminho
caminho_bases = pathlib.Path("dataset")

# traz todos os arquivos
for arquivo in caminho_bases.iterdir():      # .interdir -> da uma lista q pode iterar sobre ela com todos os caminhos do path
    print(arquivo.name)
    
# importando 1 arquivo
abril2018_df = pd.read_csv(r'dataset\abril2018.csv')
display(abril2018_df)

# esse aviso diz q a coluna 62 e 87 tem 2 tipos de dados distintos - o q faz um pouco mais lento.

abril2018.csv
abril2019.csv
abril2020.csv
agosto2018.csv
agosto2019.csv
dezembro2018.csv
dezembro2019.csv
fevereiro2019.csv
fevereiro2020.csv
janeiro2019.csv
janeiro2020.csv
julho2018.csv
julho2019.csv
junho2019.csv
maio2018.csv
maio2019.csv
maio2020.csv
maro2019.csv
maro2020.csv
novembro2019.csv
novrmbro2018.csv
outubro2018.csv
outubro2019.csv
setembro2018.csv
setembro2019.csv


C:\Users\Keylla\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (62,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,14063,https://www.airbnb.com/rooms/14063,20180414160018,2018-04-14,Living in a Postcard,"Besides the most iconic's view, our apartment ...",NaN,"Besides the most iconic's view, our apartment ...",none,Best and favorite neighborhood of Rio. Perfect...,...,f,NaN,NaN,f,f,strict_14_with_grace_period,f,f,1,0.38
1,17878,https://www.airbnb.com/rooms/17878,20180414160018,2018-04-14,Very Nice 2Br - Copacabana - WiFi,Please note that special rates apply for New Y...,- large balcony which looks out on pedestrian ...,Please note that special rates apply for New Y...,none,This is the best spot in Rio. Everything happe...,...,f,NaN,NaN,t,f,strict,f,f,1,2.17
2,24480,https://www.airbnb.com/rooms/24480,20180414160018,2018-04-14,Nice and cozy near Ipanema Beach,My studio is located in the best of Ipanema. ...,The studio is located at Vinicius de Moraes St...,My studio is located in the best of Ipanema. ...,none,"The beach, the lagoon, Ipanema is a great loca...",...,f,NaN,NaN,f,f,strict,f,f,1,0.92
3,25026,https://www.airbnb.com/rooms/25026,20180414160018,2018-04-14,Beautiful Modern Decorated Studio in Copa,"Our apartment is a little gem, everyone loves ...",This newly renovated studio (last renovations ...,"Our apartment is a little gem, everyone loves ...",none,Copacabana is a lively neighborhood and the ap...,...,f,NaN,NaN,f,f,strict,t,t,3,2.17
4,31560,https://www.airbnb.com/rooms/31560,20180414160018,2018-04-14,NICE & COZY 1BDR - IPANEMA BEACH,This nice and clean 1 bedroom apartment is loc...,This nice and clean 1 bedroom apartment is loc...,This nice and clean 1 bedroom apartment is loc...,none,Die Nachbarschaft von Ipanema ist super lebend...,...,f,NaN,NaN,t,f,strict,f,f,1,2.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39738,24393697,https://www.airbnb.com/rooms/24393697,20180414160018,2018-04-15,1 bedroom in Botafogo,"1 bedroom apartment, with a double mattress, g...",NaN,"1 bedroom apartment, with a double mattress, g...",none,NaN,...,f,NaN,NaN,t,f,flexible,f,f,2,NaN
39739,24393826,https://www.airbnb.com/rooms/24393826,20180414160018,2018-04-14,Apartamento em Copacabana,Apartamento privado em mobiliado em Copacabana...,NaN,Apartamento privado em mobiliado em Copacabana...,none,NaN,...,f,NaN,NaN,f,f,flexible,f,f,2,NaN
39740,24394510,https://www.airbnb.com/rooms/24394510,20180414160018,2018-04-14,"Apartamento 2 quartos, frente mar COPACABANA","Ampla sala, cozinha, 2 quartos, banheiro, vist...",NaN,"Ampla sala, cozinha, 2 quartos, banheiro, vist...",none,NaN,...,f,NaN,NaN,t,f,flexible,f,f,2,NaN
39741,24395755,https://www.airbnb.com/rooms/24395755,20180414160018,2018-04-15,Apto maayan,"Apartamento super confortável com wifi, TV a cabo",NaN,"Apartamento super confortável com wifi, TV a cabo",none,NaN,...,f,NaN,NaN,t,f,flexible,f,f,1,1.00


### Consolidar Base de Dados

### Se tivermos muitas colunas, já vamos identificar quais colunas podemos excluir

### Tratar Valores Faltando

### Verificar Tipos de Dados em cada coluna

### Análise Exploratória e Tratar Outliers

### Encoding

### Modelo de Previsão

### Análise do Melhor Modelo

### Ajustes e Melhorias no Melhor Modelo